In [4]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
# importing grid search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# import our models
#Logistic Regression
from sklearn.linear_model import LogisticRegression
#linear SVC
from sklearn.svm import LinearSVC
#random forest
from sklearn.ensemble import RandomForestClassifier
#xgboost
from xgboost import XGBClassifier


import os
#importing time to measure the time of execution
import time


plt.style.use("ggplot")

In [5]:
#%pip install xgboost

In [6]:
# Loading our data
data_base = pd.read_pickle('data_base.pkl')
data_base.head()

,date,home_team,away_team,season,home_win,home_rolling_fg,home_rolling_fga,home_rolling_fg%,home_rolling_3p,home_rolling_3pa,...,away_rolling_opponent_drb%_max,away_rolling_opponent_trb%_max,away_rolling_opponent_ast%_max,away_rolling_opponent_stl%_max,away_rolling_opponent_blk%_max,away_rolling_opponent_tov%_max,away_rolling_opponent_usg%_max,away_rolling_opponent_ortg_max,away_rolling_opponent_drtg_max,away_rolling_opponent_Total
0,2021-01-08,LAL,CHI,2020-21,1,44.000000,86.000000,0.510333,12.166667,31.500000,...,37.683333,25.166667,43.016667,5.083333,8.016667,47.266667,33.933333,193.000000,121.000000,119.333333
1,2021-01-09,MIL,CLE,2020-21,1,45.166667,88.666667,0.509833,16.666667,39.333333,...,37.966667,25.133333,48.700000,7.500000,6.566667,56.333333,31.833333,176.000000,109.500000,101.333333
2,2021-01-10,BRK,OKC,2020-21,0,44.666667,90.333333,0.493333,13.166667,36.166667,...,43.883333,25.016667,60.833333,5.333333,5.883333,47.533333,34.033333,169.833333,113.166667,106.500000
3,2021-01-10,GSW,TOR,2020-21,1,41.333333,87.500000,0.470833,14.000000,38.166667,...,44.950000,26.050000,48.866667,5.050000,12.916667,56.100000,32.033333,187.500000,123.166667,115.166667
4,2021-01-11,WAS,PHO,2020-21,1,44.000000,91.833333,0.479167,12.500000,34.833333,...,45.833333,25.966667,40.816667,4.550000,8.533333,38.566667,38.783333,208.333333,119.833333,108.166667


In [7]:
info_cols = ['date',
 'home_team',
 'away_team',
 'season',
 'home_win' ]


Creating our list of information columns. Note that these variables are all present before a game actually happens. 

In [8]:
# Splitting the data into train and test
X = data_base.loc[:, ~data_base.columns.isin(info_cols)] # excluding info_cols
y = data_base['home_win']

print(f'Feature set shape: {X.shape}, target series shape: {y.shape}')

Feature set shape: (2027, 264), target series shape: (2027,)


We need to perform Feature Scaling when we are dealing with Gradient Descent Based algorithms (Linear and Logistic Regression, Neural Network) and Distance-based algorithms (KNN, K-means, SVM) as these are very sensitive to the range of the data points.

In [9]:
#importing the data scaling and pipeline libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

#Creating a class that initializes the model and the parameters, and we just need to pass the scaling method for the pipeline

class ModelParams:
        """ Info:
         This class initializes the parameters and scalers for the models, which will be used for gridsearchcv,
         and their scaling methods.
          Input:
           model: the model to be used
            scaler: Default = False, if True, the model will be scaled accordily with its respective scaler
          Output:
           get_pipe: returns the model pipeline and its parameters
           model_name: returns the model name
             """
                
        #models_list = [LogisticRegression(random_state=42), LinearSVC(random_state=42), RandomForestClassifier(random_state=42), XGBClassifier(random_state=42)]     
    
        #creating the models  parameters dictionary
        # Parameters of pipelines can be set using '__' separated parameter names:
        
        models_params = {
            'LogisticRegression': {'logisticregression__penalty': ['l1', 'l2', 'elasticnet'],
                                      'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
                                        'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                                          'logisticregression__max_iter': [10000]},
            'LinearSVC': {'linearsvc__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'linearsvc__loss': ['hinge', 'squared_hinge'],
                             'linearsvc__max_iter': [10000]},
            'RandomForestClassifier': {'randomforestclassifier__bootstrap': [True, False],
                                        'randomforestclassifier__n_estimators': [100, 200, 300, 400, 500, 1000, 1500],
                                          'randomforestclassifier__criterion':['gini', 'entropy', 'log_loss'],
                                         'randomforestclassifier__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                                           'randomforestclassifier__min_samples_split': [2, 5, 10],
                                             'randomforestclassifier__min_samples_leaf': [None, 1, 2, 4],
                                         'randomforestclassifier__max_features': ['sqrt', 'log2', 'auto'],
                                           'randomforestclassifier__min_weight_fraction_leaf': [0.0, 0.4, None],
                                         'randomforestclassifier__max_leaf_nodes': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                                           'randomforestclassifier__min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0]},
            'XGBClassifier': {'xgbclassifier__n_estimators': [100, 200, 300, 400, 500], 'xgbclassifier__max_depth': [None, 5, 10, 15, 20, 25, 30],
                                 'xgbclassifier__learning_rate': [0.001, 0.01, 0.1, 1], 'xgbclassifier__min_child_weight': [1, 3, 5],
                                   'xgbclassifier__gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'xgbclassifier__subsample': [None, 0.6, 0.8, 1.0],
                                     'xgbclassifier__colsample_bytree': [None, 0.6, 0.8, 1.0], 'xgbclassifier__reg_alpha': [0, 0.001, 0.005, 0.01, 0.05, 1.5]}
        }

        #creating the models respective scalers:
        models_scalers = {
                'LogisticRegression': StandardScaler(),
                'LinearSVC': StandardScaler(),
                'RandomForestClassifier': MinMaxScaler(),
                'XGBClassifier': MinMaxScaler()
        }
        
        #creating the models respective pipelines and its parameters
        def __init__(self, model, scaler=False):
                """ Info:
                 This method initializes the model and the scaling method
                  ---------------------------------------------------------------------------------------------

                   Input:
                    model: the model to be used
                     Scaler: boolean
                      -------------------------------------------------------------------------------------------
                      
                       Output:
                        None """
                
                self.model = model
                self.scaler = scaler
                self.model_name = model.__class__.__name__

        def get_pipe(self):
                """ Info:
                 This method returns the model pipeline and its parameters and the model name
                  ---------------------------------------------------------------------------------------------

                   Input:
                    None
                    ---------------------------------------------------------------------------------------------

                     Output:
                      model pipeline, model parameters, model name """
                
                self.params = self.models_params[self.model_name]
                if self.scaler:
                        self.scaler = self.models_scalers[self.model_name]
                else:
                        self.scaler = None
                return make_pipeline(self.scaler, self.model), self.params, self.model_name
                
               
        
        


       


              
      
       

In [22]:

# creating a class to take in the model and return the metrics of this model with the best parameters,
# using for this the GridSearchCV function
class ModelDevelopment:
    """ Takes in the model, the X and y data and splits the data into train and test sets.
    Functions:
    - grid_search: Takes in the parameters to be tested and the scoring metric and returns the best model, best parameters and best score
    - model_metrics: Prints the accuracy, precision, recall, f1 and auc of best model parameters found by the grid search against the test set
    - roc_curve: Plots the ROC curve of the best model parameters found by the grid search against the test set"""

    def __init__(self, model, model_name, X, y):
        """ Info:
            Takes in the model, the X and y data and splits the data into train and test sets.

            Input:
            model: Model to be tested
            model_name: Name of the model to be tested
            X: Feature set
            y: Target series
            
            

            Output:
            X_train: Feature set for the train set
            X_test: Feature set for the test set
            y_train: Target series for the train set
            y_test: Target series for the test set
            
                    """
        self.model = model
        self.model_name = model_name
        self.best_model = None
        self.X = X
        self.y = y
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
        

        
    

    def grid_search(self, params: dict, scoring: str): # scoring is the metric we want to optimize, mostly 'roc_auc'
        """ Info:
            Takes in the parameters to be tested and the scoring metric and returns the best model, best parameters and best score
             Input:
              params: Dictionary with the parameters to be tested
              scoring: Metric to be optimized
             Output:
              best_model: Best model found by the grid search
              best_params: Best parameters found by the grid search
              best_score: Best score found by the grid search """
        
        #measuring the time it takes to run the grid search
        #start time
        start = time.time()
        
        grid = GridSearchCV(self.model, params, cv=5, scoring=scoring, n_jobs=-1, verbose=3) #verbose = 2 so we can watch the progress in more detail
        #trying to fit the model with the parameters
        try:
            grid.fit(self.X, self.y)
        except Exception as e:
            print("An error occurred:", e)
            pass
        self.best_model = grid.best_estimator_
        self.best_params = grid.best_params_
        self.best_score = grid.best_score_

        #final time
        end = time.time()
        #print the total time it took to run the grid search in minutes
        print('Total time: ', round((end-start)/60), ' minutes')

        return self.best_model, self.best_params, self.best_score
    
    def random_search(self, params_range: dict, scoring: str):
        """ Info:
            Takes in the parameters to be tested using the RandomizedSearchCV() funtion and the scoring metric and returns the best model, best parameters and best score. The randomized search allows us to test more parameters in less time.
            --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
             Input:
              params: Dictionary with the parameters to be tested
              scoring: Metric to be optimized
            --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
             Output:
              best_model: Best model found by the grid search
              best_params: Best parameters found by the grid search
              best_score: Best score found by the grid search """
        
        #measuring the time it takes to run the grid search
        #start time
        start = time.time()
        
        grid = RandomizedSearchCV(self.model, params_range, cv=3, scoring=scoring, n_jobs=-1, verbose=3, n_iter=1000 random_state=42)
        #trying to fit the model with the parameters
        try:
            grid.fit(self.X, self.y)
        except Exception as e:
            print("An error occurred:", e)
            pass
        self.best_model = grid.best_estimator_
        self.best_params = grid.best_params_
        self.best_score = grid.best_score_
        #final time
        end = time.time()
        #print the total time it took to run the grid search in minutes
        print('Total time: ', round((end-start)/60), ' minutes')
        
        
    
    def model_metrics(self):
        """ Prints the accuracy, precision, recall, f1 and auc of best model parameters found by the grid search against the test set
            For this, we use the pre split X test and y test data, which are 20% of the original data
          """
        y_pred = self.best_model.predict(self.X_test) #type: ignore
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred)
        recall = recall_score(self.y_test, y_pred)
        f1 = f1_score(self.y_test, y_pred)
        auc = roc_auc_score(self.y_test, y_pred)
        print(f'Accuracy: {accuracy:.1%}' )
        print(f'Precision: {precision:.1%}')
        print(f'Recall: {recall:.1%}')
        print(f'F1: {f1:.1%}')
        print(f'AUC: {auc:.1%}') #auc score
        
    
    def roc_curve(self):
        """ Plots the ROC curve of the best model parameters found by the grid search against the test set
            For this, we use the pre split X test and y test data, which are 20% of the original data
          """
        y_pred_proba = self.best_model.predict_proba(self.X_test)[:,1] #type: ignore
        fpr, tpr, thresholds = roc_curve(self.y_test, y_pred_proba)
        plt.plot([0,1], [0,1], 'k--')
        plt.plot(fpr, tpr, label=self.model_name)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(self.model_name+' ROC Curve')
        plt.show()

In [21]:
#creating a function that uses the ModelParams class to instantiates the model, its parameters and its respective scaler. Then, it instantiates the ModelDevelopment class and 
#uses the grid_search function to find the best parameters for the model.
def instantiate_best_model(model, X, y, scoring, scaler=True, random_search=False):
        """ Info:
                returns the model tunned with the best parameters
            -------------------------------------------------
             Input:
                model: Model to be tested
                X: Feature set
                y: Target series
                scoring: Metric to be optimized
                scaler: Whether to use a scaler or not
             ------------------------------------------------
             Output:
               best_model: Best model found by the grid search
               best_params: Best parameters found by the grid search
               best_score: Best score found by the grid search """
        
        # Using the ModelParams class to instantiate the model, its parameters and its respective scaler
        model_params = ModelParams(model, scaler)
        pipe, params, model_name = model_params.get_pipe()
        #checking if the model is already saved in the models folder
        # if the model is not saved, we instantiate the ModelDevelopment class and use the grid_search function to find the best parameters for the model and then save it
        if scaler:
                if not os.path.exists('models/'+model_name+'.pkl'): 
                         # Using the ModelDevelopment class to instantiate the model
                        clf_instance = ModelDevelopment(pipe, model_name, X, y)
                        # Using the search function to find the best parameters for the model
                        if random_search:
                                best_model, best_params, best_score  = clf_instance.random_search(params, scoring)
                        else:
                                best_model, best_params, best_score  = clf_instance.grid_search(params, scoring) # This also fits the model instance with a variety of parameters against the dataset
                        #saving best model with best parameters to pickle file:
                        file_name = 'models/'+model_name+'.pkl'
                        instance_name = 'models/'+model_name+'_instance.pkl'
                        with open(file_name, 'wb') as f:
                                pickle.dump(best_model, f)
                        with open(instance_name, 'wb') as f:
                                pickle.dump(clf_instance, f) 
                else:
                        pass
                       
        else:
                if not os.path.exists('models/'+model_name+'_unscaled.pkl'):
                         # Using the ModelDevelopment class to instantiate the model
                        clf_instance = ModelDevelopment(pipe, model_name, X, y)
                        # Using the grid_search function to find the best parameters for the model
                        best_model, *_  = clf_instance.grid_search(params, scoring) # This also fits the model instance with a variety of parameters against the dataset
                        #saving best model with best parameters to pickle file:
                        file_name = 'models/'+model_name+'_unscaled.pkl'
                        instance_name = 'models/'+model_name+'_unscaled_instance.pkl'
                        with open(file_name, 'wb') as f:
                                pickle.dump(best_model, f)
                        with open(instance_name, 'wb') as f:
                                pickle.dump(clf_instance, f) 
                else:
                        pass     
                                     

## Creating our best models objects with scaled data and saving it with pickle file, With all the propertys included with the class we created, model develpment.

### Because emsemble models will have naturally more parameters, and are more complex, we are going to set our search method to random search on these models

For this, we are going to create a function that takes in a model, the respective scaler. We are going to train either with the scaler on and off.

models_list = [LogisticRegression(random_state=42), LinearSVC(random_state=42), RandomForestClassifier(random_state=42), XGBClassifier(random_state=42)]

### Scaled data


Creating the best Logistic Regression model

In [57]:
instantiate_best_model(LogisticRegression(random_state=42), X, y, 'roc_auc', scaler=True)
        

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\joaot\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty

Total time:  2  minutes


In [12]:
LR_instance = pickle.load(open('models/LogisticRegression_instance.pkl', 'rb'))
LR_instance.model_metrics()

Accuracy: 76.8%
Precision: 77.3%
Recall: 82.7%
F1: 79.9%
AUC: 76.1%


In [16]:
LR_instance.best_params

{'logisticregression__C': 0.001,
 'logisticregression__max_iter': 10000,
 'logisticregression__penalty': 'l2',
 'logisticregression__solver': 'sag'}

Creating the best SVM model

In [60]:
instantiate_best_model(LinearSVC(random_state=42), X, y, 'roc_auc', scaler=True)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Total time:  1  minutes


In [61]:
SVM_instance = pickle.load(open('models/LinearSVC_instance.pkl', 'rb'))
SVM_instance.model_metrics()

Accuracy: 77.1%
Precision: 79.3%
Recall: 79.6%
F1: 79.5%
AUC: 76.8%


Creating the best RandomForest model


In [ ]:
instantiate_best_model(RandomForestClassifier(random_state=42), X, y, 'roc_auc', scaler=True, random_search=True)

In [ ]:
forest_instance = pickle.load(open('models/RandomForestClassifier_instance.pkl', 'rb'))
forest_instance.model_metrics()

Creating the best XGBoost model

In [ ]:
instantiate_best_model(XGBClassifier(random_state=42), X, y, 'roc_auc', scaler=True, random_search=True)

In [ ]:
xgb_instance = pickle.load(open('models/XGBClassifier_instance.pkl', 'rb'))
xgb_instance.model_metrics()

### Unscaled data

Best logistic regression model

In [63]:
instantiate_best_model(LogisticRegression(random_state=42), X, y, 'roc_auc', scaler=False)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\joaot\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\joaot\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty

Total time:  14  minutes


In [64]:
LR_instance_unscaled = pickle.load(open('models/LogisticRegression_unscaled_instance.pkl', 'rb'))
LR_instance_unscaled.model_metrics()

Accuracy: 75.4%
Precision: 76.2%
Recall: 81.0%
F1: 78.5%
AUC: 74.7%


Best svm model

In [65]:
instantiate_best_model(LinearSVC(random_state=42), X, y, 'roc_auc', scaler=False)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Total time:  3  minutes


c:\Users\joaot\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [66]:
svm_instance_unscaled = pickle.load(open('models/LinearSVC_unscaled_instance.pkl', 'rb'))
svm_instance_unscaled.model_metrics()

Accuracy: 78.1%
Precision: 79.4%
Recall: 81.9%
F1: 80.6%
AUC: 77.6%


Best random forest model

In [ ]:
instantiate_best_model(RandomForestClassifier(random_state=42), X, y, 'roc_auc', scaler=False, random_search=True)

In [ ]:
forest_instance_unscaled = pickle.load(open('models/RandomForestClassifier_unscaled_instance.pkl', 'rb'))
forest_instance_unscaled.model_metrics()

Best XGBoost model

In [ ]:
instantiate_best_model(XGBClassifier(random_state=42), X, y, 'roc_auc', scaler=False, random_search=True)

In [ ]:
xgb_instance_unscaled = pickle.load(open('models/XGBClassifier_unscaled_instance.pkl', 'rb'))
xgb_instance_unscaled.model_metrics()